## Problem 3: How long distance individuals have travelled? (8 points)

In this problem the aim is to calculate the "distance" in meters that the individuals have travelled according the social media posts (Euclidean distances between points). In this problem, we will need the `userid` -column and the points created in the previous problem. You will need the shapefile `Kruger_posts.shp` generated in Problem 2 as input file.

Our goal is to answer these questions based on the input data:
- What was the shortest distance travelled in meters?
- What was the mean distance travelled in meters?
- What was the maximum distance travelled in meters?

**In your code, you should first:**
 - Import required modules.
 - Read in the shapefile as a geodataframe called `data`
 - Reproject the data from WGS84 projection into `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into the metric system.
 
*Store the result in a variable called `data`*!

In [1]:
# Import required modules
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from shapely.geometry import Point, LineString, Polygon

import warnings
warnings.filterwarnings('ignore')

# Read in the shapefile
fp = "Kruger_posts.shp"
data = gpd.read_file(fp)

- Check the crs of the input data. If this information is missing, set it as epsg:4326 (WGS84).
- Reproject the data from WGS84 to `EPSG:32735` -projection which stands for UTM Zone 35S (UTM zone for South Africa) to transform the data into metric system. (don't create a new variable, update the existing variable `data`!)"

In [2]:
# Check the crs
if not data.crs:
    data.crs = CRS.from_epsg(4326).to_wkt()
    
# Reproject the data
data.to_crs(epsg=32735, inplace=True)

In [3]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check the data
print(data.head())

         lat        lon         timestamp    userid  \
0 -24.980792  31.484633  2015-07-07 03:02  66487960   
1 -25.499225  31.508906  2015-07-07 03:18  65281761   
2 -24.342578  30.930866  2015-03-07 03:38  90916112   
3 -24.854614  31.519718  2015-10-07 05:04  37959089   
4 -24.921069  31.520836  2015-10-07 05:19  27793716   

                            geometry  
0  POINT (-4695752.719 14973674.275)  
1  POINT (-4748939.258 15014098.837)  
2  POINT (-4672729.591 14859391.193)  
3  POINT (-4679391.656 14969037.444)  
4  POINT (-4686373.982 14973910.589)  


In [4]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

# Check that the crs is correct after re-projecting (should be epsg:32735)
print(data.crs)

epsg:32735


 - Group the data by userid

In [5]:
# Group the data
grouped = data.groupby("userid")

In [6]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the number of groups:
assert len(grouped.groups) == data["userid"].nunique(), "Number of groups should match number of unique users!"

**Create LineString objects for each user connecting the points from oldest to latest:**

*Suggested steps:*
- Create an empty DataFrame called `movements`. 
- Create an empty column "geometry"
- Use a for-loop where you iterate over the grouped object. For each user's data: 
    - [sort](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html) the rows by timestamp 
    - create a LineString object based on the user's points
    - Add the LineString to the geometry column of the `movements` dataframe. You can also add the `userid` in a separate column (or use the userid as index).
- Convert `movements` into a `GeoDataFrame` (you can replace the DataFrame created in the previous steps with the GeoDataFrame). Remember to set the `geometry` column.
- Set the CRS of the ``movements`` GeoDataFrame as ``EPSG:32735`` 

In [7]:
# Create empty Dataframe
movements = pd.DataFrame(columns=["userid", "geometry"])

# Add Shapely LineString geometry objects to the DataFrame
for key, group in grouped:
    if len(group) > 1:
        group.sort_values(by="timestamp", inplace=True)
        movements = movements.append({"userid": key, 
                                      "geometry": LineString(group["geometry"].tolist())},
                                     ignore_index=True)
        
# Convert DataFrame into a GeoDataFrame
movements = gpd.GeoDataFrame(movements, geometry="geometry", crs=CRS.from_epsg(32735).to_wkt())

In [8]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the result
print(type(movements))
print(movements.crs)
print(movements["geometry"].head())

<class 'geopandas.geodataframe.GeoDataFrame'>
PROJCRS["WGS 84 / UTM zone 35S",BASEGEOGCRS["WGS 84",ENSEMBLE["World Geodetic System 1984 ensemble",MEMBER["World Geodetic System 1984 (Transit)"],MEMBER["World Geodetic System 1984 (G730)"],MEMBER["World Geodetic System 1984 (G873)"],MEMBER["World Geodetic System 1984 (G1150)"],MEMBER["World Geodetic System 1984 (G1674)"],MEMBER["World Geodetic System 1984 (G1762)"],MEMBER["World Geodetic System 1984 (G2139)"],ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1]],ENSEMBLEACCURACY[2.0]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433]],ID["EPSG",4326]],CONVERSION["UTM zone 35S",METHOD["Transverse Mercator",ID["EPSG",9807]],PARAMETER["Latitude of natural origin",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8801]],PARAMETER["Longitude of natural origin",27,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["Scale factor at natural origin",0.9996,SCALEUNIT["unity",1],ID["EPSG",8805]],PARAMETER["False e

**Finally:**
- Check once more the crs definition of your dataframe (should be epsg:32735, define the correct crs if this information is missing)
- Calculate the lenghts of the lines into a new column called ``distance`` in ``movements`` GeoDataFrame.

In [9]:
# Check the crs
if not movements.crs:
    data.crs = CRS.from_epsg(32735).to_wkt()
    
    
# Calculate the lengths    
movements["distance"] = movements.apply(lambda row: row["geometry"].length, axis=1)

In [10]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

#Check the output
movements.head()

,userid,geometry,distance
0,16301,"LINESTRING (-4681550.088 14943799.279, -468323...",454640.989781
1,45136,"LINESTRING (-4770692.230 14940874.449, -477069...",0.000000
2,50136,"LINESTRING (-4680731.675 14947431.176, -468798...",205712.690132
3,88775,"LINESTRING (-4773713.345 14938272.132, -477371...",0.095916
4,88918,"LINESTRING (-4699374.159 14988142.858, -468798...",11388.305140


You should now be able to print answers to the following questions: 

 - What was the shortest distance travelled in meters?
 - What was the mean distance travelled in meters?
 - What was the maximum distance travelled in meters?

In [11]:
# What was the shortest distance travelled in meters?
print(f"The shortest distance travelled was: {movements['distance'].min()} m.")

# What was the mean distance travelled in meters?
print(f"The mean distance travelled was: {round(movements['distance'].mean(), 1)} m.")

# What was the maximum distance travelled in meters?
print(f"The maximum distance travelled was: {movements['distance'].max()} m.")

The shortest distance travelled was: 0.0 m.
The mean distance travelled was: 138871.1 m.
The maximum distance travelled was: 8457917.497356469 m.


- Finally, save the movements of into a Shapefile called ``some_movements.shp``

In [12]:
# Save the GeoDataFrame to a Shapefile
fp = "some_movements.shp"
movements.to_file(fp)

In [13]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION

import os

#Check if output file exists
assert os.path.isfile(fp), "Output file does not exits."

That's all for this week!